# download libs
pip install torch transformers accelerate bitsandbytes langchain langchain-community langchain-experimental langchain-huggingface langchain-chroma langchain-text-splitters langchain-core chromadb

In [28]:
import torch
torch.cuda.is_available()

True

In [29]:
from transformers import BitsAndBytesConfig # for compressing model e.g. 16bits -> 4bits

from transformers import (
                          AutoTokenizer, # Tokenize Model
                          AutoModelForCausalLM,  # LLM Loader - used for loading and using pre-trained models designed for causal language modeling tasks
                          pipeline) # pipline to setup llm-task oritented model
                                    # pipline("text-classification", model='model', device=0)

from langchain_huggingface import HuggingFaceEmbeddings # huggingface sentence_transformer embedding models
from langchain_huggingface.llms import HuggingFacePipeline # like transformer pipeline

from langchain.memory import ConversationBufferMemory # Deprecated
from langchain_community.chat_message_histories import ChatMessageHistory # Deprecated
from langchain_community.document_loaders import PyPDFLoader, TextLoader # PDF Processing
from langchain.chains import ConversationalRetrievalChain # Deprecated
from langchain_experimental.text_splitter import SemanticChunker # module for chunking text

from langchain_chroma import Chroma # AI-native vector databases (ai-native mean built for handle large-scale AI workloads efficiently)
from langchain_text_splitters import RecursiveCharacterTextSplitter # recursively divide text, then merge them together if merge_size < chunk_size
from langchain_core.runnables import RunnablePassthrough # Use for testing (make 'example' easy to execute and experiment with)
from langchain_core.output_parsers import StrOutputParser # format LLM's output text into (list, dict or any custom structure we can work with)
from langchain import hub

In [30]:
from huggingface_hub import notebook_login
notebook_login()

In [31]:
# Read PDF file
Loader = PyPDFLoader
# FILE_PATH = "25 Thuật Ngữ AI - Machine Learning dễ hiểu cho người mới.pdf"
FILE_PATH = "iot_security_report.pdf"
loader = Loader(FILE_PATH)
documents = loader.load()

In [32]:
print(documents[:50])

[Document(metadata={'producer': 'pdfTeX-1.40.26', 'creator': 'LaTeX with hyperref', 'creationdate': '2025-06-11T15:24:02+00:00', 'author': '', 'keywords': '', 'moddate': '2025-06-11T15:24:02+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.26 (TeX Live 2024) kpathsea version 6.4.0', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'iot_security_report.pdf', 'total_pages': 35, 'page': 0, 'page_label': '1'}, page_content='IoT Security\nNgười thực hiện:Nguyễn Bình Nam\nNgười hướng dẫn:Trần Quang Tài\nNgày 11 tháng 6 năm 2025'), Document(metadata={'producer': 'pdfTeX-1.40.26', 'creator': 'LaTeX with hyperref', 'creationdate': '2025-06-11T15:24:02+00:00', 'author': '', 'keywords': '', 'moddate': '2025-06-11T15:24:02+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.26 (TeX Live 2024) kpathsea version 6.4.0', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'iot_security_report.pdf', 'total_pages': 35, 'page': 1, 'page_label'

[bkai-foundation-model 2024](https://huggingface.co/bkai-foundation-models/vietnamese-bi-encoder)

In [33]:
embeddings = HuggingFaceEmbeddings(
    model_name = "bkai-foundation-models/vietnamese-bi-encoder",
    model_kwargs = {'device': 'cuda'},
    encode_kwargs = {'normalize_embeddings': True}
) # convert text to vector (not chunking yet)

In [34]:
# runtime:
# + bkai-foundation-models/vietnamese-bi-encoder: 3 mins
# + keepitreal/vietnamese-sbert: 3mins
semantic_splitter = SemanticChunker(
    embeddings=embeddings,
    buffer_size=1, # total sentence collected before perform text split
    breakpoint_threshold_type='percentile', # set splitting style: 'percentage' of similarity
    breakpoint_threshold_amount=95, # split text if similarity score > 95%
    min_chunk_size=500,
    add_start_index=True, # assign index for chunk
)

docs = semantic_splitter.split_documents(documents)
print("Number of sementic chunks:", len(docs))

Number of sementic chunks: 50


In [35]:
vector_db = Chroma.from_documents(documents=docs,
                                embedding=embeddings)

retriever = vector_db.as_retriever()

In [36]:
result = retriever.invoke("IoT là gì ?")
print("Num of relevant documents: ", len(result))

#? Không Embedd được hình (ý nghĩa của hình)
#? May retrieve duplicate documents
for i, doc in enumerate(result, 1):
    print(f"\n📄 Documellmnt {i}")
    print("-" * 60)
    print(f"📄 Page       : {doc.metadata.get('page_label', doc.metadata.get('page'))}")
    print(f"📝 Content    :\n{doc.page_content.strip()}")
    print("-" * 60)

Num of relevant documents:  4

📄 Documellmnt 1
------------------------------------------------------------
📄 Page       : 4
📝 Content    :
Chương 1
Tổng quan về IoT
1.1 IoT là gì? Internet of Things (IoT) là một hệ thống gồm các thiết bị được gọi là "vật". Những thiết bị này có các bộ phận điện tử cho phép chúng có khả năng
tự động thu thập dữ liệu thông qua cảm biến, xử lý và trao đổi dữ liệu
qua mạng. Những thiết bị này có thể là các thiết bị gia dụng đơn giản như
router hay các thiết bị phức tạp như máy móc công nghiệp quy mô lớn
Đặc trưng nổi bật của IoT là tính liên thông năng lượng thấp: mỗi
thiết bị IoT sẽ luôn luôn được bật và đều được kết nối và tương tác với các
thiết bị IoT khác trong cùng hệ thống. Những giao thức kết nối được sử
dụng trong hệ thống IoT cần được tối ưu cho hệ thống năng lượng thấp và
ít tài nguyên tính toán. Một đặc trưng khác của những thiết bị IoT này là
các thiết bị này có khả năng xử lý và thu thập thông tin trong thời gian
thực. 1.2 Các thành phần tro

In [37]:
with open('token.txt', 'r') as f:
    hg_token = f.read() #? read huggingface token from token.txt file

In [38]:
# set up config
nf4_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [48]:
#? Initialize Model and Tokenizer
#? PhoGPT-5.5B
#? Phi-2 (2.7B)
#? lmsys/vicuna-7b-v1.5
MODEL_NAME= "google/gemma-2b-it"

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=nf4_config, # add config
    low_cpu_mem_usage=True,
    token=hg_token
).to("cuda")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [49]:
# Check if the model is on CUDA
if next(model.parameters()).is_cuda:
    print("Model is running on CUDA.")
else:
    print("Model is not running on CUDA.")

Model is running on CUDA.


In [50]:
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME,
    use_fast=True,
    padding_side='left',   # 'left' or 'right' depending on model style (e.g., causal LM often prefers left)
    truncation_side='left'
)

In [51]:
tokenizer.pad_token = tokenizer.eos_token

# #? Integrated tokenizer and model into a Pipeline (for convinient)
model_pipeline = pipeline(
    'text-generation',
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024, # output token
    device_map="auto" # auto allocate GPU if available
)

llm = HuggingFacePipeline(
    pipeline=model_pipeline,
)

Device set to use cuda:0


## Learn how to prompt so the LLM can generate better multiple-choice question

Ví dụ về một câu hỏi trắc nghiệm tốt:

Câu hỏi: Tấn công side-channel là gì?

Phương án:

A. Là tấn công từ xa vào giao diện web.

B. Là kiểu tấn công dựa trên hành vi tiêu thụ năng lượng của thiết bị.

C. Là tấn công trực diện vào hạ tầng mạng

D. Là tấn công dựa vào bức xạ điện từ để lấy khóa mã hóa.

Đáp án đúng: D

In [52]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

parser = StrOutputParser()


In [53]:
import os

def save_result(result, file_path):
    with open(file_path, 'w') as f:
            f.write(result)

In [54]:
from langchain.prompts import PromptTemplate

In [55]:
prompt = PromptTemplate.from_template("""
        Trả lời ngắn gọn, rõ ràng bằng tiếng việt và chỉ dựa trên thông tin có sẵn bên dưới.
        Nếu không tìm thấy thông tin, hãy nói rõ là không có dữ liệu liên quan.

        Nội dung tài liệu:
        {context}

        Câu hỏi:
        {question}

        Trả lời:
    """) #? dùng {{ }} để langchain không nhận string bên trong {} là Biến

rag_chain = (
{"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | parser
)

query = 'Liệt kê các thành phần trong hệ thống IoT ?'
result = rag_chain.invoke(query)

In [56]:
print(result)


        Trả lời ngắn gọn, rõ ràng bằng tiếng việt và chỉ dựa trên thông tin có sẵn bên dưới.
        Nếu không tìm thấy thông tin, hãy nói rõ là không có dữ liệu liên quan.

        Nội dung tài liệu:
        Chương 1
Tổng quan về IoT
1.1 IoT là gì? Internet of Things (IoT) là một hệ thống gồm các thiết bị được gọi là "vật". Những thiết bị này có các bộ phận điện tử cho phép chúng có khả năng
tự động thu thập dữ liệu thông qua cảm biến, xử lý và trao đổi dữ liệu
qua mạng. Những thiết bị này có thể là các thiết bị gia dụng đơn giản như
router hay các thiết bị phức tạp như máy móc công nghiệp quy mô lớn
Đặc trưng nổi bật của IoT là tính liên thông năng lượng thấp: mỗi
thiết bị IoT sẽ luôn luôn được bật và đều được kết nối và tương tác với các
thiết bị IoT khác trong cùng hệ thống. Những giao thức kết nối được sử
dụng trong hệ thống IoT cần được tối ưu cho hệ thống năng lượng thấp và
ít tài nguyên tính toán. Một đặc trưng khác của những thiết bị IoT này là
các thiết bị này có khả năng xử

### Customize RAG Output to Json

In [ ]:
multi_choice_prompt_improved = """
Bạn là một trợ lý chuyên xử lý văn bản và chỉ trả lời bằng định dạng JSON.
Dựa vào Context được cung cấp, hãy thực hiện các nhiệm vụ sau và trả về kết quả trong một đối tượng JSON duy nhất.

NHIỆM VỤ:
1. Rút ra tối đa 3 ý chính từ Context có liên quan trực tiếp đến Câu hỏi. Mỗi ý phải có nguồn (số trang) đi kèm.
2. Trả lời Câu hỏi một cách ngắn gọn và chính xác.

QUY TẮC BẮT BUỘC:
- Câu trả lời của bạn PHẢI là một đối tượng JSON hợp lệ. KHÔNG thêm bất kỳ văn bản, giải thích, hay ghi chú nào trước hoặc sau khối JSON.
- Nếu không tìm thấy ý chính nào liên quan, hãy trả về một danh sách rỗng cho trường `main_ideas`, ví dụ: `"main_ideas": []`.
- Nếu Context không chứa thông tin để trả lời Câu hỏi, hãy điền chuỗi "Không có dữ liệu liên quan" vào trường `answer`.

CẤU TRÚC JSON BẮT BUỘC:
{format_instructions}

DỮ LIỆU ĐẦU VÀO:
Context:
{context}

Câu hỏi:
{question}

JSON Output:
"""


In [ ]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.schema.runnable import Runnable, RunnableSequence, RunnableMap,RunnableLambda

# Define individual Runnables
sentiment_analysis_runnable = RunnableLambda(lambda text: "Positive" if "good" in text.lower() else "Negative") # check if the word "good" exist inside text string

summarization_runnable = PromptTemplate(input_variables=["text"], template="Summarize this: {text}") | llm # | represent RunnableSequence


# Combine Runnables into a pipeline
pipeline = RunnableMap({
    "sentiment": sentiment_analysis_runnable,
    "summary": summarization_runnable
})

# Invoke the pipeline
feedback = "The product quality is really nice and exceeded expectations."
result = pipeline.invoke(feedback)

print(result)
# Output:
# {
#   "sentiment": "Positive",
#   "summary": "The product quality is excellent."
# }

{'sentiment': 'Negative', 'summary': 'Summarize this: The product quality is really nice and exceeded expectations. \n\nThe customer service was excellent and attentive.\n\nThe packaging was well-designed and protected the product.\n\nOverall, the customer had a positive experience with the product and the company.'}


In [ ]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List

class MainIdea(BaseModel):
    point: str
    source: str

# return output as pre-define object format
class QAResponse(BaseModel):
    main_ideas: List[MainIdea]
    answer: str

# converting raw input into QAResponse Object data structure
parser = PydanticOutputParser(pydantic_object=QAResponse)
prompt_template = PromptTemplate(
    #? use to instruct the llm using prompt
    template=multi_choice_prompt_improved,
    #? specifies information (input_variable) the llm would received
    input_variables=["context", "question"],
    #? inject the formatting instructions generate by the parser
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

#? format type: LangChain Expression Language (LCEL)
#?
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt_template
    | llm
    | parser
)

# query = 'Liệt kê các thành phần trong hệ thống IoT ?'
query = 'IoT là gì ?'
try:
    result = rag_chain.invoke(query)
    print(result)
except Exception as e:
    print(f"There a Error: {e}")

In [ ]:
multi_choice_prompt = """
        Dựa vào nội dung sau, hãy:
        1. Tóm tắt tối đa 3 ý chính, kèm theo số trang nếu có.
        2. Trả lời câu hỏi bằng tiếng Việt ngắn gọn và chính xác.
        3. Nếu không có thông tin liên quan, hãy để "Answer" là "Không có dữ liệu liên quan".

        Đảm bảo trả kết quả **ở dạng JSON** với cấu trúc sau:
        {{"main_ideas": [
            {{"point": "Ý chính 1", "source": "Trang ..."}},
            {{"point": "Ý chính 2", "source": "Trang ..."}},
            {{"point": "Ý chính 3", "source": "Trang ..."}}
        ],
        "answer": "Câu trả lời ngắn gọn"
        }}

        Vui lòng chỉ trả lời bằng format JSON, không giải thích thêm.

        Context:
        {context}

        Question:
        {question}

        Answer:

""" #? dùng {{ }} để langchain không nhận string bên trong {} là Biến

In [ ]:
def run_custom_rag(user_question):
    prompt = PromptTemplate.from_template(multi_choice_prompt)
    rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | llm
        | parser
    )


    query = user_question
    result = rag_chain.invoke(query)

    file_path = 'output.txt'
    save_result(result, file_path)


    return result

In [ ]:
question = "IoT là gi?"
result = run_custom_rag(question)

OutputParserException: Invalid json output: Dựa vào nội dung sau, hãy:
        1. Tóm tắt tối đa 3 ý chính, kèm theo số trang nếu có.
        2. Trả lời câu hỏi bằng tiếng Việt ngắn gọn và chính xác.
        3. Nếu không có thông tin liên quan, hãy để "Answer" là "Không có dữ liệu liên quan".

        Đảm bảo trả kết quả **ở dạng JSON** với cấu trúc sau:
        {"main_ideas": [
            {"point": "Ý chính 1", "source": "Trang ..."},
            {"point": "Ý chính 2", "source": "Trang ..."},
            {"point": "Ý chính 3", "source": "Trang ..."}
        ],
        "answer": "Câu trả lời ngắn gọn"
        }

        Vui lòng chỉ trả lời bằng format JSON, không giải thích thêm.

        Context:
        Chương 1
Tổng quan về IoT
1.1 IoT là gì? Internet of Things (IoT) là một hệ thống gồm các thiết bị được gọi là "vật". Những thiết bị này có các bộ phận điện tử cho phép chúng có khả năng
tự động thu thập dữ liệu thông qua cảm biến, xử lý và trao đổi dữ liệu
qua mạng. Những thiết bị 

In [ ]:
query = "Các thành phần trong hệ thống IoT bao gồm những gì ?"
result = run_custom_rag(query)